In [127]:
import torch
import sys
from torch.utils.data import DataLoader
import torchvision.transforms as T
from tqdm import tqdm
import numpy as np
from mean_average_precision import MetricBuilder

sys.path.append('../src/')
from newspapersdataset import (
    NewspapersDataset, 
    prepare_data_for_dataloader
)
from functions import (
    from_tsv_to_list,
    collate_fn
)

sys.path.append("../../")
from image_size import get_image_size  # source: https://github.com/scardine/image_size

In [128]:
model = torch.load('../saved_models/model.pth', map_location=torch.device('cpu'))

In [129]:
parameters = {
    'channel': 1, 
    'rescale': [1000, 1000],
    'batch_size': 16,
    'shuffle': False, 
    'num_workers': 2,
    'main_dir': '/Users/alexdrozdz/Desktop/Studia/00. Seminarium magisterskie/Master_degree/',
    'image_dir': '/Users/alexdrozdz/Desktop/Studia/00. Seminarium magisterskie/scraped_photos_final/',
    'annotations_dir': '/Users/alexdrozdz/Desktop/Studia/00. Seminarium magisterskie/news-navigator/',
}

In [130]:
data_transform = T.Compose([
    T.Grayscale(num_output_channels=parameters['channel']),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,)),
    ])

In [131]:
# prepare data 
expected = from_tsv_to_list(parameters['annotations_dir']+'dev-0/expected.tsv')
in_file = from_tsv_to_list(parameters['annotations_dir']+'dev-0/in.tsv')
img_paths = [parameters['image_dir']+path for path in in_file]

data = prepare_data_for_dataloader(
    img_dir=parameters['image_dir'],
    in_list=in_file,
    expected_list=expected,
    bbox_format='x0y0x1y1',
    scale=parameters['rescale'],
    test=False,
    )
dataset = NewspapersDataset(
    df=data,
    images_path=img_paths,
    scale=parameters['rescale'],
    transforms=data_transform,
    test=False,
    )
dataloader = DataLoader(
    dataset,
    batch_size=parameters['batch_size'],
    shuffle=parameters['shuffle'],
    collate_fn=collate_fn,
    num_workers=parameters['num_workers'],
    )

In [132]:
# predict on cpu
device = 'cpu'
model.to(device)
torch.set_num_threads(1)
cpu_device = torch.device('cpu')

In [133]:
# predict on the test set
with torch.no_grad():
    f_out, f_tar = [], []
    for images, targets in tqdm(dataloader):
        images = list(img.to(cpu_device) for img in images)
        targets = [{k: v.to(cpu_device) for k, v in t.items()} for t in targets]
        f_tar.append(targets)
        outputs = model(images)
        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        f_out.append(outputs)

 86%|████████▌ | 12/14 [19:06<03:03, 91.60s/it]

In [ ]:
f_out

In [ ]:
f_tar

In [104]:
# prepare results
pred_list, gt_list = [], []
for i in tqdm(range(len(f_out))):
    for ii in range(len(f_out[i])):
        for iii in range(len(f_out[i][ii]['boxes'])):
            # prediction
            temp_list_pred = list(f_out[i][ii]['boxes'].detach().numpy()[iii])
            temp_list_pred.append(f_out[i][ii]['labels'].detach().numpy()[iii])
            temp_list_pred.append(f_out[i][ii]['scores'].detach().numpy()[iii])
            pred_list.append(temp_list_pred)
            # ground truth
            temp_list_gt = list(f_tar[i][ii]['boxes'].detach().numpy()[iii])
            temp_list_gt.append(f_tar[i][ii]['labels'].detach().numpy()[iii])
            temp_list_gt.append([0, 0])
            pred_list.append(temp_list_gt)

final_pred_arr, final_gt_arr = np.array(pred_list), np.array(gt_list)

100%|██████████| 45/45 [00:00<00:00, 94.86it/s]


In [ ]:
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=True, num_classes=7)

for i in range(len(pred_list)):
    metric_fn.add(final_pred_arr, final_gt_arr)

In [ ]:
print(f"mAP: {metric_fn.value(iou_thresholds=np.arange(0.5, 1.0, 0.05), recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy='soft')['mAP']}")

In [112]:
# # prepare results
# final_list = []
# for i in tqdm(range(len(f_out))):
#     for ii in range(len(f_out[i])):
#         img_width, img_height = get_image_size.get_image_size(
#                 parameters['image_dir'] + str(int(f_tar[i][ii]['image_name'].detach().numpy())) + '.jpg'
#             )
#         scaler_width = np.float(img_width)/np.float(f_tar[i][ii]['new_image_size'].detach().numpy()[0][0])
#         scaler_height = np.float(img_height)/np.float(f_tar[i][ii]['new_image_size'].detach().numpy()[0][1])
#         for iii in range(len(f_out[i][ii]['boxes'])):
#             pred_boxes = f_out[i][ii]['boxes'].detach().numpy()[iii]
#             x0 = int(round(pred_boxes[0]*scaler_width,0))
#             y0 = int(round(pred_boxes[1]*scaler_height,0))
#             x1 = int(round(pred_boxes[2]*scaler_width,0))
#             y1 = int(round(pred_boxes[3]*scaler_height,0))
#             temp_list = [x0, y0, x1, y1]
#             temp_list.append(f_out[i][ii]['labels'].detach().numpy()[iii])
#             temp_list.append(f_out[i][ii]['scores'].detach().numpy()[iii])
#             final_list.append(temp_list)

# final_arr = np.array(final_list)

array([[5.09000000e+02, 2.78000000e+02, 1.23800000e+03, 8.79000000e+02,
        6.00000000e+00, 9.99558628e-01],
       [2.61100000e+03, 4.34000000e+02, 3.98000000e+03, 8.84000000e+02,
        6.00000000e+00, 9.99542117e-01],
       [1.23000000e+03, 2.81000000e+02, 1.92000000e+03, 1.01700000e+03,
        6.00000000e+00, 9.99483943e-01],
       ...,
       [2.88200000e+03, 2.54300000e+03, 3.52200000e+03, 2.67400000e+03,
        6.00000000e+00, 9.46213678e-02],
       [9.08000000e+02, 4.45000000e+02, 2.23600000e+03, 2.41800000e+03,
        5.00000000e+00, 7.02385679e-02],
       [2.21700000e+03, 3.72000000e+02, 2.88500000e+03, 8.49000000e+02,
        7.00000000e+00, 6.86615780e-02]])